# Should reorder a bit notebook and clean it, and write basis of pipeline

1. Construct `stop_times_zurich` (now it uses the graph to get nodes, but unecessary), cache it
2. Planner, when user asks journey:
    1. Create graph (2 hours before arrival)
    2. Create 2nd graph with only stations and edges reachable from Zürich
    3. Compute paths
    4. Find shortest path

## 1. Setup:

First we setup our spark application and load the necessary data containing information about specific transit information:  
- `stop_times.txt` : arrival and departure times at stops
- `stops.txt` : information about stops 
- `trips.txt` : information about journeys
- `calendar.txt` : information about which servcies are active on which dates

#### Set up spark:

In [1]:
%%configure
{"conf": {
    "spark.app.name": "dslab-group_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4896,application_1587988164357_1532,pyspark,idle,Link,Link,
4916,application_1587988164357_1552,pyspark,idle,Link,Link,
4957,application_1587988164357_1594,pyspark,idle,Link,Link,
4968,application_1587988164357_1605,pyspark,idle,Link,Link,
4970,application_1587988164357_1607,pyspark,idle,Link,Link,
4971,application_1587988164357_1608,pyspark,idle,Link,Link,
4973,application_1587988164357_1610,pyspark,idle,Link,Link,
4974,application_1587988164357_1611,pyspark,idle,Link,Link,
4975,application_1587988164357_1612,pyspark,dead,Link,Link,
4976,application_1587988164357_1613,pyspark,idle,Link,Link,


#### Imports:

In [2]:
import networkx as nx
from geopy.distance import distance as geo_distance
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
from networkx.algorithms.shortest_paths.weighted import dijkstra_path

The code failed because of a fatal error:
	Invalid status code '500' from http://iccluster044.iccluster.epfl.ch:8998/sessions with error payload: "java.io.IOException: Could not get block locations. Source file \"/livy2-recovery/v1/interactive/state.tmp\" - Aborting...block==null".

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


#### Load data:

In [3]:
# Loading data, these are snapshots of the all available data
# Calendar and trips are useful to filter the other dataframe according to the day

stop_times = spark.read.format('orc').load('/data/sbb/timetables/orc/stop_times/000000_0')
stops = spark.read.format('orc').load('/data/sbb/timetables/orc/stops/000000_0')
trips = spark.read.format('orc').load('/data/sbb/timetables/orc/trips/000000_0')
calendar = spark.read.format('orc').load('/data/sbb/timetables/orc/calendar/000000_0')

The code failed because of a fatal error:
	Invalid status code '500' from http://iccluster044.iccluster.epfl.ch:8998/sessions with error payload: "java.io.IOException: Could not get block locations. Source file \"/livy2-recovery/v1/interactive/state.tmp\" - Aborting...block==null".

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


## 2. Pre-processing: 

Here, we pre-process the data to correspond to the following criteria:
- only consider journeys at reasonable hours of the day, and on a typical business day, and assuming the schedule of May 13-17, 2019
- allow short (max 500m "As the Crows Flies") walking distances for transfers between two stations, and assume a walking speed of 50m/1min on a straight line, regardless of obstacles, human-built or natural, such as building, highways, rivers, or lakes
- only consider journeys that start and end on known station coordinates (train station, bus stops, etc.), never from a random location
- only consider stations in a 15km radius of Zürich's train station, Zürich HB (8503000), (lat, lon) = (47.378177, 8.540192)
- only consider stations in the 15km radius that are reachable from Zürich HB, either directly, or via transfers through other stations within the area
- assume that the timetables remain unchanged throughout the 2018 - 2019 period

Filtering criteria for later use (e.g. probability intervals and planner):
- assuming that delays or travel times on the public transport network are uncorrelated with one another
- once a route is computed, a traveller is expected to follow the planned routes to the end, or until it fails (i.e. miss a connection)
- planner will not need to mitigate the traveller's inconvenience if a plan fails

### Stop times during rush-hour: 

Only consider journeys at reasonable hours of the day, thus we take only stop times that are in the window of rush-hour (e.g. from 8 a.m. to 8 p.m.). 

In [ ]:
# Filter stop_times to be only in 08:00-19:59:
stop_times = stop_times.where((col('departure_time') >= '08:00:00') 
                              & (col('departure_time') <= '19:59:59'))

### Stations around Zürich HB:

Only consider stations in a 15km radius of Zürich's train station (Zürich HB). 

First we get the geolocation of Zürich Hauptbahnhof to be able to calculate the distance of the other stations to the Hauptbahnhof. 

In [ ]:
zurich_pos = stops.where(column('stop_name') == 'Zürich HB').select('stop_lat', 'stop_lon').collect()
zurich_pos = (zurich_pos[0][0], zurich_pos[0][1])
print('Location of Zürich Hauptbahnhof (lat, lon) :'+str(zurich_pos))

In [ ]:
def zurich_distance(x, y):
    """zurich_distance: returns the distance of a station to Zurich HB
    @input: (lat,lon) of a station
    @output: distance in km to Zurich HB
    """
    return geo_distance(zurich_pos, (x,y)).km

Then we create a dataframe `stops_zurich` of the stations where we add a column for the distance to Zurich HB. In that dataframe, we keep only those that are in a radisu of 15km to the HB. The same filter is applied to the `stop_times` df mentioned above. 

In [ ]:
# filter stops:
stops_distance = stops.rdd.map(lambda x: (x['stop_id'], zurich_distance(x['stop_lat'], x['stop_lon'])))
stops_distance = spark.createDataFrame(stops_distance.map(lambda r: Row(stop_id=r[0], 
                                                                        zurich_distance=r[1])))

stops_distance = stops_distance.filter(column('zurich_distance') <= 15)

print('There are '+str(stops_distance.count())+' stops in a radius of 15km around Zurich HB')

# add distance to HB to stops info and keep only in radius of 15km
stops_zurich = stops_distance.join(stops, on='stop_id')

# keep only stop times in radius of 15km of Zurich
stop_times = stop_times.join(stops_distance.select('stop_id'), on='stop_id')

**bizarre**: je voulais prendre les trips dans le rayon de 15km seulement mais bizarrement ça fait un dataframe plus grand que trips,... 

In [ ]:
"""
trips_zurich = trips.join(stop_times.select('trip_id').distinct().count(), on = 'trip_id', how = 'inner')
trips_zurich.count()
"""

In [ ]:
# keep only trips in radius of 15km of Zurich
#trips_zurich = trips.join(stop_times.select('trip_id').distinct(), on = 'trip_id', how = 'inner')

# WEIRD : why is trips_zurich.count() > trips.count() ? 

In [ ]:
# Cache it to save time:
stop_times.cache()

### Have a look at the data we have so far: 

#### Stop times in Zurich: 

In [ ]:
stop_times.show(3)

#### Trips in Zurich:

In [ ]:
trips.show(3)

#### Stops in Zurich:

In [ ]:
stops_zurich.show(3)

#### Calendar:

In [ ]:
calendar.show(3)

## 3. Create a network:

From the pre-processed data, we would like to create a network where each node is a station and each edge between two nodes corresponds to a possible trip. 

A node will have the following attributes:
- stop_name: name of the station (e.g. Zurich HB)
- latitude
- longitude

An edge will have the following attributes:
- dd

First, we create a function that returns all services that operate on certain days (as some of them might not operate on weekends for example). 

In [ ]:
days_dict = {0: 'monday', 1: 'tuesday', 2: 'wednesday', 3: 'thursday', 4: 'friday'}

def day_trips(*day_ids):
    """
    day_trips: gives the trip_ids that operate on certain days
    input: ? 
    output:s spark dataframe with trip_ids
    
    """
    days = [days_dict[day_id] for day_id in day_ids]
    where_clause = " and ".join(days)

    day_services = calendar.where(where_clause).select('service_id')
    return day_services.join(trips, on='service_id').select('trip_id')

Then we create a **multigraph** (e.g. more than one edge allowed between two nodes). 

In [ ]:
graph = nx.MultiDiGraph()

nodes = stops_zurich.rdd.map(lambda r: (r[0], {'name': r['stop_name'],
                                              'lat': r['stop_lat'],
                                              'lon': r['stop_lon']})).collect()
graph.add_nodes_from(nodes)

In [ ]:
@udf
def convertToMinute(s):
    h, m, _ = s.split(':')
    h,m = int(h), int(m)
    
    return h*60+m

# Keep only travels around zurich
nodes_list = list(graph.nodes())
stop_times_zurich = stop_times.filter(column('stop_id').isin(nodes_list))
print('Number of stop times around zurich:', stop_times_zurich.count())
# Convert time information to minutes elapsed since 0am
stop_times_zurich = stop_times_zurich.withColumn('arrival_time', convertToMinute(column('arrival_time')))
stop_times_zurich = stop_times_zurich.withColumn('departure_time', convertToMinute(column('departure_time')))
# Add next stop to dataframe
stop_times_zurich_2 = (stop_times_zurich.withColumn('stop_sequence_prev', column('stop_sequence')-1)
                       .select('trip_id',
                               column('stop_id').alias('next_stop'),
                               column('stop_sequence_prev').alias('stop_sequence'),
                               column('arrival_time').alias('next_arrival_time')))
# Add trip duration
stop_times_zurich = stop_times_zurich.join(stop_times_zurich_2, on=['trip_id', 'stop_sequence']).orderBy('trip_id', 'stop_sequence')
stop_times_zurich = stop_times_zurich.withColumn('trip_duration', column('next_arrival_time')-column('departure_time'))
stop_times_zurich = stop_times_zurich.select('trip_id', 'stop_id', 'arrival_time', 'departure_time', 'next_stop', 'trip_duration')
stop_times_zurich.cache()

In [ ]:
max_trip_duration = 2 #duration in hour 
def filter_edge_on_time(edges_df, day_id, hour, minute):
    edges_df = edges_df.join(day_trips(day_id), on='trip_id')
    arrival_minute = hour*60+minute
    min_dep_time = arrival_minute - 60*60*max_trip_duration
    edges_df = edges_df.filter((col('departure_time') > min_dep_time) & 
                                            (col('arrival_time') <= arrival_minute))

    return edges_df

# Example of graph construction: Wednesday arrival at 11:30:00
edges = (filter_edge_on_time(stop_times_zurich, 2, 11, 30)
         .rdd.map(lambda r: (r['stop_id'], r['next_stop'], {'duration': r['trip_duration'],
                                                          'time': float(r['departure_time'])})).collect())
print('Number of edges:', len(edges))

In [ ]:
_ = graph.add_edges_from(edges)

In [ ]:
stop_ids_zurich = stops_zurich.where(col('stop_name') == 'Zürich HB').select('stop_id').rdd.flatMap(lambda x: x).collect()
stop_ids_zurich

In [ ]:
nodes_reachable_from_zurich = set(stop_ids_zurich)

for id_ in stop_ids_zurich:
    nodes_reachable_from_zurich = nodes_reachable_from_zurich.union(nx.descendants(graph, id_))
    
new_edges = [e for e in graph.edges(data=True) if e[0] in nodes_reachable_from_zurich and e[1] in nodes_reachable_from_zurich]

In [ ]:
graph_zurich = nx.MultiDiGraph()
graph_zurich.add_nodes_from(nodes_reachable_from_zurich)
_ = graph_zurich.add_edges_from(new_edges)

In [ ]:
nx.descendants(graph_zurich, '8503000:0:10')

In [ ]:
dijkstra_path(graph_zurich, '8503000:0:10', '8502208:0:3', weight='duration')

In [ ]:
gen = nx.all_simple_paths(graph_zurich, '8503000:0:10', '8502208:0:3')

for path in gen:
    print(path, flush=True)

In [ ]:
graph.out_edges('8503000:0:10', data=True)

In [ ]:
def filter_edges(edges, time):
    edges = [edge for edge in edges if edge[2]['time'] >= time]
    
    destinations = set([edge[1] for edge in edges])
    earliest_edges = []
    
    for destination in destinations:
        edges_to_dest = [edge for edge in edges if edge[1] == destination]
        earliest_edge = sorted(edges_to_dest, key=lambda edge: edge[2]['time'] + edge[2]['duration'])[0]
        earliest_edges.append(earliest_edge)
    
    return earliest_edges

In [ ]:
def get_paths(graph, source, target, time, prev_path=[], num_hops=0, max_num_hops=4):
    if source == target:
        return [prev_path]
    elif num_hops >= max_num_hops:
        return None
    else:
        out_edges = graph.out_edges(source, data=True)
        out_edges = filter_edges(out_edges, time)
        # Should manually check
        print(time)
        print(out_edges)
        paths = []
        
        for out_edge in out_edges:
            new_paths = get_paths(graph, out_edge[1], target, time=out_edge[2]['time']+out_edge[2]['duration'], prev_path=prev_path+[source], num_hops=num_hops+1)
            if new_paths is not None:
                paths = paths + new_paths
                    
        return paths

In [ ]:
out_edges = graph.out_edges('8503006:0:6', data=True)
out_edges = [edge for edge in out_edges]

In [ ]:
out_edges[0]

In [ ]:
dijkstra_path(graph_zurich, '8503000:0:10', '8503011:0:1')

In [ ]:
nx.descendants(graph_zurich, '8503000:0:10')

In [ ]:
get_paths(graph_zurich, '8503000:0:10', '8503202:0:4', 570, max_num_hops=6)

In [ ]:
# Example for stop_times filtered on wednesday
stop_times_wed = day_trips(2).join(stop_times, on='trip_id')
stop_times_wed.show(5)

stop_times_wed.count()

In [ ]:
# Can't run, the count makes it timeout. I asked Tao why
#print('Full stop times have', stop_times.count(), 'entries, filtered has', stop_times_wed.count())

In [ ]:
#for r in stops_zurich.collect():
#    if r['stop_id'] == '8503000':
#        print((r['stop_id'], {'name': r['stop_name'].encode('utf-8'), 'lat': r['stop_lat'], 'lon': r['stop_lon']}))
#        break